https://news.ycombinator.com/item?id=23770180

Majromax on July 8, 2020 | parent | context | favorite | on: SymPy - a Python library for symbolic mathematics

I like SymPy as a light-weight CAS. I don't have any heavy symbolic computational needs, but it fills a gap between "simple enough to work out with pen and paper, without error" and "so complicated that it's best to stick to numerical simulation."

However, as a slightly-more-than-beginner user I have a frustrating time getting SymPy to take simplifications that I know are valid. As a result the generated answers are correct but so complicated that the "simple intuitive understanding" goal of the symbolic computation is lost.

This can come down to having implicit assumptions that I haven't told SymPy (such as variables being real-valued, or it being okay to take the principal roots of exp(I <stuff>)), but not always.

As a minimal example:

In [2]:
  import sympy as sp # Import SymPy

In [3]:
  # Define variables; note they're marked as real-valued
  x = sp.Symbol('x', real=True);
  t = sp.Symbol('t', real=True);

In [4]:
  # Get our simple expression: 2*cos(t)*exp(x) but written in complex form
  foo = sp.exp((x+sp.I*t)) + sp.exp((x-sp.I*t))
  print(str(foo))

exp(-I*t + x) + exp(I*t + x)


In [12]:
# Get our simple expression: 2*cos(t)*exp(x) but written in complex form
foo = sp.exp((x+sp.I*t)) + sp.exp((x-sp.I*t))
foo

exp(-I*t + x) + exp(I*t + x)

In [5]:
  # Prints exp(-I*t + x) + exp(I*t + x)
  print(str(foo.simplify()))

(exp(2*I*t) + 1)*exp(-I*t + x)


In [13]:
# Prints exp(-I*t + x) + exp(I*t + x)
foo.simplify()

(exp(2*I*t) + 1)*exp(-I*t + x)

In [6]:
  # Still prints exp(-I*t + x) + exp(I*t + x) -- SymPy thinks this is the simplest form?

In [7]:
  print(str((foo/sp.exp(x)).simplify())) # Help it along
  # Prints 2*cos(t)

2*cos(t)


In [14]:
(foo/sp.exp(x)).simplify()

2*cos(t)

 Obviously, forcing SymPy to do the desired thing gets worse with more complicated expressions. Sometimes I can bash it into compliance with liberal use of `rewrite` or by demanding it simplify term-by-term, but if I'm deconstructing the expression only to reassemble it at the end then I'm not sure I'm still gaining much out of it as a computer algebra system.
 

ogogmad on July 8, 2020 [–]

For your particular example, try foo.rewrite(cos). If you work with trigonometric expressions, it often pays to randomly do foo.rewrite(cos) or foo.rewrite(exp) along with simplify().

Your larger point is true though, at least from my experience. 

In [15]:
foo.rewrite(sp.cos)

-I*sin(I*(-I*t + x)) - I*sin(I*(I*t + x)) + cos(I*(-I*t + x)) + cos(I*(I*t + x))

In [16]:
foo.rewrite(sp.cos).simplify()

2*exp(x)*cos(t)

 Majromax on July 8, 2020 | parent [–]

Alas, sp.rewrite doesn't quite do what I want for the slightly more complicated (with new symbol 'k' suitably defined):

  bar = sp.exp(sp.I*(x + k - t)) - sp.exp(sp.I*(x+k+t))

The desired output would be 2 cos(t) exp(I(x+k)), but rewrite and simplify works on an all-or-nothing basis, affecting exp(I(x+k)) as well.

My mental framework is to come at this from something like a signal engineering perspective, looking at amplitude-modulated single frequencies. However, expressing that intent in a way that SymPy understands is for now beyond me.

My problem may ultimately be one of documentation. The basic SymPy tutorials are quite nice, and the raw documentation for the functions... exists, but I have yet to find good examples of intermediate to advanced use discussing the "what and why" of accomplishing nontrivial goals. 

In [17]:
k = sp.Symbol('k', real = True)
bar = sp.exp(sp.I*(x + k - t)) - sp.exp(sp.I*(x+k+t))

In [18]:
bar

exp(I*(k - t + x)) - exp(I*(k + t + x))

In [19]:
bar.simplify()

exp(I*(k - t + x)) - exp(I*(k + t + x))

In [21]:
bar.rewrite(sp.exp)

exp(I*(k - t + x)) - exp(I*(k + t + x))

In [22]:
bar.rewrite(sp.cos)

I*sin(k - t + x) - I*sin(k + t + x) + cos(k - t + x) - cos(k + t + x)

In [25]:
bar.rewrite(sp.cos).simplify()

exp(I*(k - t + x)) - exp(I*(k + t + x))